# Bokeh and HoloViews Visualizations

This notebook demonstrates interactive visualizations with Bokeh and HoloViews.

**Libraries:**
- [Bokeh](https://bokeh.org/) - Interactive visualization for web browsers
- [HoloViews](https://holoviews.org/) - Declarative data visualization
- [hvPlot](https://hvplot.holoviz.org/) - High-level plotting API

In [ ]:
import numpy as np
import pandas as pd

# Bokeh imports
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.palettes import Category10

# HoloViews imports
import holoviews as hv
from holoviews import opts

# Enable notebook output
output_notebook()
hv.extension("bokeh")

## Bokeh Basics

Bokeh provides elegant, interactive visualizations for modern web browsers.

### Interactive Scatter Plot with Hover

In [ ]:
np.random.seed(42)
n = 100
x = np.random.randn(n)
y = 2 * x + np.random.randn(n) * 0.5
colors = ["blue" if val > 0 else "red" for val in y]
sizes = np.abs(y) * 5 + 5

# Create data source
source = ColumnDataSource(data=dict(x=x, y=y, colors=colors, sizes=sizes))

# Create figure with tools
p = figure(
    title="Interactive Scatter Plot (hover over points)",
    x_axis_label="X",
    y_axis_label="Y",
    tools="pan,wheel_zoom,box_zoom,reset,hover,save",
    width=700,
    height=400,
)

p.scatter("x", "y", color="colors", size="sizes", alpha=0.6, source=source)

# Configure hover tool
hover = p.select_one(HoverTool)
hover.tooltips = [("X", "@x{0.2f}"), ("Y", "@y{0.2f}")]

show(p)

### Multiple Line Plot with Legend

In [ ]:
x_line = np.linspace(0, 4 * np.pi, 100)

p = figure(
    title="Multiple Time Series (click legend to hide/show)",
    x_axis_label="Time",
    y_axis_label="Value",
    width=700,
    height=400,
)

for i, (label, func) in enumerate([
    ("sin", np.sin),
    ("cos", np.cos),
    ("tan (clipped)", lambda x: np.clip(np.tan(x), -3, 3))
]):
    p.line(x_line, func(x_line), legend_label=label, line_color=Category10[3][i], line_width=2)

p.legend.location = "top_left"
p.legend.click_policy = "hide"  # Click legend items to hide/show

show(p)

### Bar Chart with Hover Tooltips

In [ ]:
categories = ["Product A", "Product B", "Product C", "Product D", "Product E"]
values = [28, 55, 43, 91, 67]
colors = Category10[5]

source = ColumnDataSource(data=dict(categories=categories, values=values, colors=colors))

p = figure(
    x_range=categories,
    title="Sales by Product (hover for details)",
    tools="hover,save",
    width=700,
    height=400,
)

p.vbar(x="categories", top="values", width=0.8, color="colors", source=source)
p.hover.tooltips = [("Product", "@categories"), ("Sales", "@values")]
p.xaxis.major_label_orientation = 0.5

show(p)

## Bokeh Dashboard Layout

Combine multiple plots into a dashboard.

In [ ]:
# Create multiple plots for dashboard
np.random.seed(42)
dates = pd.date_range("2024-01-01", periods=50, freq="D")
values = np.cumsum(np.random.randn(50)) + 100

# Time series
p1 = figure(title="Time Series", x_axis_type="datetime", width=350, height=250)
p1.line(dates, values, line_width=2)
p1.circle(dates, values, size=4)

# Histogram
hist, edges = np.histogram(np.random.randn(1000), bins=30)
p2 = figure(title="Distribution", width=350, height=250)
p2.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], alpha=0.7)

# Scatter
p3 = figure(title="Correlation", width=350, height=250)
x = np.random.randn(100)
y = x * 0.7 + np.random.randn(100) * 0.3
p3.scatter(x, y, alpha=0.6)

# Bar chart
p4 = figure(title="Comparison", width=350, height=250, x_range=["A", "B", "C"])
for i, (cat, val) in enumerate([("A", 20), ("B", 35), ("C", 15)]):
    p4.vbar(x=[cat], top=[val], width=0.5, color=Category10[3][i])

# Layout: 2x2 grid
layout = column(row(p1, p2), row(p3, p4))
show(layout)

## HoloViews Declarative Visualization

HoloViews lets you build visualizations declaratively by describing your data.

In [ ]:
# Create sample DataFrame
np.random.seed(42)
df = pd.DataFrame({
    "x": np.random.randn(200),
    "y": np.random.randn(200),
    "category": np.random.choice(["A", "B", "C"], 200),
    "value": np.random.rand(200) * 100,
})
df["y"] = df["y"] + df["category"].map({"A": 0, "B": 2, "C": 4})
df.head()

### HoloViews Scatter Plot

In [ ]:
scatter = hv.Scatter(df, kdims=["x"], vdims=["y", "category", "value"])
scatter.opts(
    opts.Scatter(color="category", cmap="Category10", size=8, tools=["hover"], width=600, height=400)
)

### HoloViews Curve Overlay

In [ ]:
x_vals = np.linspace(0, 2 * np.pi, 100)
curve_sin = hv.Curve([(x, np.sin(x)) for x in x_vals], label="sin(x)")
curve_cos = hv.Curve([(x, np.cos(x)) for x in x_vals], label="cos(x)")

overlay = (curve_sin * curve_cos).opts(
    opts.Curve(width=600, height=400, tools=["hover"])
)
overlay

### HoloViews Heatmap

In [ ]:
heatmap_data = [(i, j, np.sin(i) * np.cos(j)) for i in range(10) for j in range(10)]
heatmap = hv.HeatMap(heatmap_data).opts(
    opts.HeatMap(colorbar=True, width=500, height=400, cmap="viridis", tools=["hover"])
)
heatmap

### HoloViews Layout

In [ ]:
bars = hv.Bars([("A", 10), ("B", 20), ("C", 15)]).opts(opts.Bars(width=300, height=300))
hist = hv.Histogram(np.histogram(np.random.randn(1000), bins=30)).opts(
    opts.Histogram(width=300, height=300)
)

layout = (bars + hist).opts(shared_axes=False)
layout

## hvPlot - Pandas Integration

hvPlot provides a familiar `.plot()` API for Pandas DataFrames.

In [ ]:
import hvplot.pandas  # Adds .hvplot to pandas objects

### Time Series with hvPlot

In [ ]:
dates = pd.date_range("2024-01-01", periods=100, freq="D")
ts_df = pd.DataFrame({
    "date": dates,
    "value1": np.cumsum(np.random.randn(100)),
    "value2": np.cumsum(np.random.randn(100)),
})
ts_df = ts_df.set_index("date")

ts_df.hvplot.line(title="Time Series with hvPlot", width=700, height=400)

### Scatter with hvPlot

In [ ]:
df.hvplot.scatter(
    x="x",
    y="y",
    c="category",
    s="value",
    title="Scatter Plot with hvPlot",
    width=700,
    height=400,
)

---

## Summary

In this notebook, we covered:

1. **Bokeh Basics**: Scatter plots, line plots, bar charts with interactivity
2. **Bokeh Layouts**: Dashboard-style multi-plot layouts
3. **HoloViews**: Declarative scatter, curve, heatmap, and layouts
4. **hvPlot**: Pandas-integrated interactive plotting

For more information:
- [Bokeh Gallery](https://docs.bokeh.org/en/latest/docs/gallery.html)
- [HoloViews Gallery](https://holoviews.org/gallery/)
- [hvPlot Documentation](https://hvplot.holoviz.org/)